In [1]:
%%time
import time

from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import matplotlib.pyplot as plt
import umap.umap_ as umap
from sklearn.datasets import make_blobs
import numpy as np
%matplotlib inline
import scipy
import time
import networkx as nx
import collections
import EoN
from collections import defaultdict
import random
import pandas as pd


sns.set(style='white', context='poster', rc={'figure.figsize':(8,6)})
#default_seed = 16


CPU times: user 4.28 s, sys: 810 ms, total: 5.09 s
Wall time: 50.3 s


In [ ]:
%%time
start = time.time()
s = 1250
arr = [int(0.2*s),int(0.05*s),int(0.1*s),int(0.16*s),int(0.12*s),int(0.1*s),int(0.04*s),int(0.08*s),int(0.15*s)]
data,y = make_blobs(n_samples=arr,n_features=2,random_state=16, cluster_std=1)

time1= time.time()
print("Data creation: ",time1-start)

r = np.random.RandomState(10)

coo_matrix1 = umap.fuzzy_simplicial_set(data,30,r,'euclidean')
coo_matrix1 = np.asarray(coo_matrix1)

time2 = time.time()
print("Fuzzy simplicial:",time2-time1,"   ||| From start: ",time2-start)

#repeat
sparse_matrix = coo_matrix1[0] 
#print(sparse_matrix.getnnz())

G = nx.from_scipy_sparse_matrix(sparse_matrix)

time3 = time.time()
print("Graph formation:",time3-time2,"   ||| From start: ",time3-start)

print(f'number of edges BEFORE reducing superspreader connections: {G.number_of_edges()}') 
ss = sorted(G.degree, key=lambda x: x[1], reverse=True)
k1=round(2/100 * len(ss))  ## top n percent of highest degree nodes   ##############################################################################################
ss1= [i[0] for i in ss]
ss_final = ss1[:k1]

for i in ss_final:
    k2=round(80/100 * len(G.edges(i)))  ## removing random 80% edges/connections of each selected superspreader
    list1 = list(G.edges(i))
    to_remove=random.sample(list1,k2)
    G.remove_edges_from(to_remove)
    
time4 = time.time()
print(f'number of edges AFTER reducing superspreader connections: {G.number_of_edges()}') 
print("reducing superspreader connections (80%):",time4-time3,"   ||| From start: ",time4-start)

fig, ax = plt.subplots()
my_pos = nx.spring_layout(G,seed =13)    ###########################################################################################################################
nx.draw_networkx(G,pos= my_pos,width=0.1,with_labels=False, node_size=6, node_color=y,cmap = 'viridis',edge_color='black',)
ax.axis("off")
plt.savefig("./results/ssfinal_graph_2per")    #####################################################################################################################

H1 = nx.DiGraph()
H1.add_node('S')
H1.add_edge('E', 'I', rate = 0.7)
H1.add_edge('I', 'R', rate = 0.3)

J1 = nx.DiGraph()
J1.add_edge(('I', 'S'), ('I', 'E'), rate = 0.5, weight_label='weight')
IC = defaultdict(lambda: 'S')
for node in range(50):
    IC[node] = 'I'
    
return_statuses = ('S', 'E', 'I', 'R')

t, S, E, I, R = EoN.Gillespie_simple_contagion(G, H1, J1, IC, return_statuses,
                                        tmax = float('inf'))

save =pd.DataFrame(list(zip(t,S,E,I,R)), columns =['t','S','E','I','R'])
save.to_pickle('ss.pkl')

baseline = pd.read_pickle("baseline.pkl")
ss = pd.read_pickle("ss.pkl")

fig, ax = plt.subplots(figsize = (16,9))
plt.semilogy(baseline['t'],baseline['S'], label = 'Susceptible',color= 'black', linewidth = 1, zorder =1, linestyle = 'dashed')
plt.semilogy(baseline['t'],baseline['I'], label = 'Susceptible',color= 'darkorange', linewidth = 1, zorder =1, linestyle = 'dashed')
plt.semilogy(baseline['t'],baseline['R'], label = 'Susceptible',color= 'forestgreen', linewidth = 1, zorder =1, linestyle = 'dashed')

plt.semilogy(ss['t'],ss['S'], label = 'Susceptible',color= 'black', linewidth = 2, zorder =2)
plt.semilogy(ss['t'],ss['I'], label = 'Susceptible',color= 'darkorange', linewidth = 2, zorder =2)
plt.semilogy(ss['t'],ss['R'], label = 'Susceptible',color= 'forestgreen', linewidth = 2, zorder =2)

ax.set_xlim(0, 600)
ax.set_ylim(1,1500000)
plt.savefig('results/ssfinal_2per')

time5 = time.time()
print("SEIR:",time5-time4,"   ||| Total: ",time5-start)